## Getting Started with Axolotl: BGC Analytics with PySpark and Bigslice
This Jupyter notebook tutorial is my personal getting started guide to utilizing [Axolotl](https://github.com/JGI-Bioinformatics/axolotl), a scalable distributed genome and metagenome data analysis for the exploration of genomic data, specifically focusing on the identification and study of BGCs, which are groups of genes involved in the biosynthesis of complex bioactive compounds. In this notebook, I am learning how to set up and manage Spark sessions, process GenBank files, and utilize Bigslice applications for efficient and scalable genomic data analysis. 

In [ ]:
from pyspark import SparkContext
from pathlib import Path
from pyspark.sql import SparkSession
from axolotl.app.bgc_analysis import BigsliceApp
from axolotl.io.genbank import gbkIO
from axolotl.data.annotation import RawFeatDF
import os, shutil

def start_new_spark_session(master_ip, app_name, log_dir="../logs", cores=2, memory="2g", partitions=5):
    """
    This function creates a new Spark session with the specified configuration.
    
    Parameters:
    master_ip (str): The IP address of the master node.
    app_name (str): The name of the Spark application.
    log_dir (str): The directory where Spark event logs will be stored. Defaults to "../logs".
    
    Returns:
    tuple: A tuple containing the SparkSession and SparkContext objects.
    """
    
    # Stop the existing SparkContext if there is one
    SparkContext.getOrCreate().stop()
    
    # Create the log directory if it doesn't exist
    log_dir = Path(log_dir)
    log_dir.mkdir(parents=True, exist_ok=True)

    # Create a new SparkSession with the specified configuration
    spark = SparkSession \
        .builder \
        .appName(app_name) \
        .master(master_ip) \
        .config("spark.executor.cores", str(cores)) \
        .config("spark.executor.memory", memory) \
        .config("spark.shuffle.useOldFetchProtocol", True) \
        .config("spark.sql.shuffle.partitions", partitions) \
        .config("spark.sql.parquet.enableVectorizedReader", False) \
        .config("spark.eventLog.enabled", True) \
        .config("spark.eventLog.dir", log_dir) \
        .getOrCreate()
    
    # Get the SparkContext from the SparkSession
    sc = spark.sparkContext
    
    # Return the SparkSession and SparkContext
    return (spark, sc)

def gbk_to_bigslice_app(genbank_table_path, bigslice_app_dir_path, annotations_path):
    """
    This function loads sequence and annotation data from datasets, and creates a BigsliceApp.

    Parameters:
    genbank_table_path (str): The path to the file containing the datasets.
    bigslice_app_dir_path (str): The path to the directory where the BigsliceApp will be created.
    annotations_path (str): The path to the file containing the annotations.
    """

    # Define the path to the file containing the datasets
    genbank_table = genbank_table_path

    # Load the annotation data from the datasets into a DataFrame
    gbkIO.loadSmallFiles2(
        str(genbank_table), 
        df_type="annotation",
        minPartitions=5, 
        skip_malformed_record=True
    ).write(
        annotations_path, 
        overwrite=True, 
        num_partitions=5
    )

    # Define the path to the directory
    bigslice_app_dir = Path(bigslice_app_dir_path)

    # Delete the existing directory if it exists
    if bigslice_app_dir.exists():
        shutil.rmtree(str(bigslice_app_dir))

    # Now you can create the BigsliceApp without getting an error
    features_df = RawFeatDF.read(annotations_path)
    BigsliceApp.create(str(bigslice_app_dir), 
        features_df, 
        source_type="antismash")

### Setting up Spark Session & Hadoop Libary (TO DO)
To begin our journey with Axolotl and Bigslice for BGC analysis, we first need to initialize a Spark session. This session will serve as the foundation for our distributed data processing tasks. By setting up the Spark session with the local machine as the master node, we enable the utilization of all available cores for parallel processing. Naming our application "bigslice" helps in identifying our Spark application processes and logs more easily. The following code snippet demonstrates how to start a new Spark session with these configurations:

In [ ]:
# Start a new Spark session with the local machine as the master node and "bigslice" as the application name
spark, sc = start_new_spark_session("local[*]", "bigslice")

### Setting up input files
Before diving into the analysis with Axolotl and Bigslice, it's crucial to gather and prepare our input data. In this case, we're focusing on GenBank (.gbk) files generated by antiSMASH, a popular tool for identifying biosynthetic gene clusters (BGCs) in genomic data. These files contain valuable annotations and sequence information that Axolotl will process.

Here's how we prepare our dataset list:

In [ ]:
# Define the path to the directory containing the datasets
dataset_path = "/home/matinnu/datadrive/vol_atlas_server/datasets/"

# Use a list comprehension to get a list of all .gbk files in the antismash subdirectories
datasets = [i for i in Path(dataset_path).glob("*/antismash/*region*.gbk")]

# Define the path to the output file
outfile = Path(genbank_table)

# Create the parent directory of the output file if it doesn't exist
outfile.parent.mkdir(parents=True, exist_ok=True)

# Open the output file in write mode
with open(outfile, 'w') as f:
    # Write the paths of the datasets to the file, one per line
    f.writelines(str(dataset) + '\n' for dataset in datasets)

### Loading Data into the BiG-SLICE App

After preparing our dataset list, the next step involves loading this data into the BiG-SLICE application for analysis. BiG-SLICE is a powerful tool designed for large-scale comparative analysis of biosynthetic gene clusters (BGCs), enabling researchers to uncover the diversity and distribution of BGCs across large genomic datasets. This section outlines the process of converting GenBank files into a format suitable for BiG-SLICE and setting up the necessary directories and databases for the analysis.

In [ ]:
# Setup where to save parquets
genbank_table = "../data/datasets.txt"
bigslice_app_dir = "../data/interim/bigslice_app"
annotations_dir = "../data/interim/annotations"

# Setup bigslice HMM database, can be downloaded with hmm
bigslice_db_path = Path(os.getenv("CONDA_PREFIX")) / "bin/bigslice-models"

# Write parquet files to bigslice app
gbk_to_bigslice_app(genbank_table, bigslice_app_dir, annotations_dir)

### Executing BiG-SLICE analysis from the app

In [ ]:
bigslice_app = BigsliceApp.load(bigslice_app_dir)

In [ ]:
bigslice_app.getData("bgc").df.show(2)

In [ ]:
bigslice_app.getData("cds").df.show(2)